## Grafiquitos

In [ ]:
import glob
import matplotlib.pyplot as plt
import numpy as np
import os.path
import pandas as pd
import seaborn as sns

In [ ]:
BENCH_DIR = '../../bench/'
GRAPH_DIR = '../../graph/'

CSV_COLUMNS = ["gyms","stops","bagSize","time","reps","status",
               "generator","result","algorithm","subtype","seed1","seed2"]

Configuración de matplotlib

In [ ]:
def plt_setlegend(ax, legend=None):
    if legend is not None:
        l = ax.legend(legend, loc='best', frameon=True, borderpad=0.5)
    else:
        l = ax.legend(loc='best', frameon=True, borderpad=0.5)
    l.get_frame().set_linewidth(1)

### Leemos los benchmarks
Separamos los de time de los de result, ya que los últimos se corren una sola vez y pueden tener mucho ruido

In [ ]:
def readDf(wildcard):
    files = glob.glob(os.path.join(BENCH_DIR, wildcard))
    dfs = (pd.read_csv(f, names=CSV_COLUMNS, header=None) for f in files)
    df = pd.concat(dfs, ignore_index=True)
    df["nodes"] = df["gyms"] + df["stops"]
    df["time"] = df["time"] / 1e9
    return df

df_time = readDf("*-time-*")
df_result = readDf("*-result-*")

### Datos de las mediciones

In [ ]:
df_time[["algorithm","subtype"]].drop_duplicates()

In [ ]:
df_time.describe()

In [ ]:
df_result.describe()

## Graficos de tiempo

In [ ]:
t_is_exacto = df_time["algorithm"] == 'exacto'
t_is_greedy = df_time["algorithm"] == 'greedy'
t_is_local = df_time["algorithm"] == 'local'
t_is_grasp = df_time["algorithm"] == 'grasp'

t_is_bruteforce = df_time["subtype"] == 'none'
t_is_backtracking = df_time["subtype"] == 'backtracking'
t_is_closest = df_time["subtype"] == 'closest'
t_is_furthest = df_time["subtype"] == 'furthest'
t_is_2opt = df_time["subtype"] == 'dos_opt'
t_is_swap = df_time["subtype"] == 'swap'

t_is_not_inf = df_time["result"] != np.inf

In [ ]:
plt.clf()
axis1 = plt.subplot(1,2,1)
plt.ylabel('Tiempo en segundos')
axis2 = plt.subplot(1,2,2)
plt.ylabel('Tiempo en segundos')

df_time[t_is_exacto & t_is_bruteforce].plot(
    x="nodes", y="time", ax=axis1,
    label="fuerza bruta", style='o')

df_time[t_is_exacto & t_is_backtracking].plot(
    x="nodes", y="time", ax=axis2,
    label="backtracking", style='go')

# Seteamos la misma escala para ambos
axis1.set_yscale('log')
axis2.set_yscale('log')
x1,x2,y1,y2 = axis1.axis()
axis2.set_ylim(y1,y2)

plt.suptitle("Runtime de los algoritmos exactos, con mochilas 'infinitas'")
plt_setlegend(axis1)
plt_setlegend(axis2)
plt.savefig(GRAPH_DIR + 'time-exacto.png')
plt.show()

In [ ]:
plt.clf()
axis1 = plt.subplot(1,2,1)
plt.ylabel('Tiempo en segundos')
axis2 = plt.subplot(1,2,2)
plt.ylabel('Tiempo en segundos')

df_time[t_is_greedy & t_is_furthest & t_is_not_inf].plot(
    x="nodes", y="time", ax=axis1,
    label="heurística greedy, closest first", style='o')
df_time[t_is_greedy & t_is_closest & t_is_not_inf].plot(
    x="nodes", y="time", ax=axis2,
    label="heurística greedy, furthest first", style='go')

# Seteamos la misma escala para ambos
x1,x2,y1,y2 = axis1.axis()
axis2.set_ylim(y1,y2)

plt.suptitle("Runtime de las heurísticas greedy, con mochilas 'infinitas'")
plt_setlegend(axis1)
plt_setlegend(axis2)
plt.savefig(GRAPH_DIR + 'time-greedy.png')
plt.show()

In [ ]:
plt.clf()
axis1 = plt.subplot(1,2,1)
plt.ylabel('Tiempo en segundos')
axis2 = plt.subplot(1,2,2)
plt.ylabel('Tiempo en segundos')

ax = df_time[t_is_local & t_is_swap & t_is_not_inf].plot(
    x="nodes", y="time", ax=axis1,
    label="heurística local, swap", style='o')
ax = df_time[t_is_local & t_is_2opt & t_is_not_inf].plot(
    x="nodes", y="time", ax=axis2,
    label="heurística local, 2opt", style='go')

# Seteamos la misma escala para ambos
x1,x2,y1,y2 = axis1.axis()
axis2.set_ylim(y1,y2)

plt.suptitle("Runtime de las heurísticas locales, con mochilas 'infinitas'")
plt_setlegend(axis1)
plt_setlegend(axis2)
plt.savefig(GRAPH_DIR + 'time-local.png')
plt.show()

In [ ]:
plt.clf()
fig, ax = plt.subplots()

ax = df_time[t_is_grasp & t_is_not_inf].plot(x="nodes", y="time", ax=ax,
                            label="heurística grasp", style='o')
plt.ylabel('Tiempo en segundos')
plt.title("Runtime de la metaheurística grasp, con mochilas 'infinitas'")
plt_setlegend(ax)
plt.savefig(GRAPH_DIR + 'time-grasp.png')
plt.show()

## Graficos de precisión

### En funcion del exacto

In [ ]:
is_exacto = df_result["algorithm"] == 'exacto'
is_greedy = df_result["algorithm"] == 'greedy'
is_local = df_result["algorithm"] == 'local'
is_grasp = df_result["algorithm"] == 'grasp'

is_closest = df_result["subtype"] == 'closest'
is_furthest = df_result["subtype"] == 'furthest'
is_2opt = df_result["subtype"] == 'dos_opt'
is_swap = df_result["subtype"] == 'swap'

merge_cols = ["gyms","stops","nodes","bagSize","generator","seed1","seed2"]
columns = merge_cols + ["result"]
ext_columns = columns + ["algorithm", "subtype"]

small_exacto = df_result[is_exacto][columns]
small_exacto = small_exacto[small_exacto["result"] != np.inf]
small_exacto.rename(columns={"result": "result_exacto"}, inplace=True)

def getPrecision(data_filter, name):
    tmp = pd.merge(df_result[data_filter][ext_columns],
                           small_exacto, on=merge_cols)
    tmp = tmp[tmp["result"] != np.inf]
    tmp["precision"] = tmp["result"] / tmp["result_exacto"]
    tmp["name"] = name
    return tmp

prec_greedy_closest = getPrecision(is_greedy & is_closest, 'greedy closest first')
prec_greedy_furthest = getPrecision(is_greedy & is_furthest, 'greedy furthest first')
prec_local_2opt = getPrecision(is_local & is_2opt, 'local, 2opt')
prec_local_swap = getPrecision(is_local & is_swap, 'local, swap')
prec_grasp = getPrecision(is_grasp, 'grasp')

prec_all = prec_greedy_closest.append(prec_greedy_furthest
                             ).append(prec_local_2opt).append(prec_local_swap
                             ).append(prec_grasp)

In [ ]:
sns.boxplot(x='name', y='precision', data=prec_all)
plt.savefig(GRAPH_DIR + 'precision-small-all.png')
plt.show()

In [ ]:
sns.boxplot(x='nodes', y='precision', data=prec_local_2opt)
plt.savefig(GRAPH_DIR + 'precision-small-local-2opt.png')
plt.show()

In [ ]:
sns.boxplot(x='nodes', y='precision', data=prec_local_swap)
plt.savefig(GRAPH_DIR + 'precision-small-local-swap.png')
plt.show()

### En funcion del mínimo entre las herísticas

In [ ]:
is_exacto = df_result["algorithm"] == 'exacto'
is_greedy = df_result["algorithm"] == 'greedy'
is_local = df_result["algorithm"] == 'local'
is_grasp = df_result["algorithm"] == 'grasp'

is_closest = df_result["subtype"] == 'closest'
is_furthest = df_result["subtype"] == 'furthest'
is_2opt = df_result["subtype"] == 'dos_opt'
is_swap = df_result["subtype"] == 'swap'

merge_cols = ["gyms","stops","nodes","bagSize","generator","seed1","seed2"]
columns = merge_cols + ["result"]
min_columns = merge_cols + ["min_result"]
ext_columns = columns + ["algorithm", "subtype"]

minResult = df_result[is_grasp].rename(columns={"result": "min_result"})
minResult = minResult[min_columns]
minResult = minResult[minResult["min_result"] != np.inf]

def mergeMinimums(data_filter):
    global minResult
    minResult = pd.merge(df_result[data_filter][columns], minResult, on=merge_cols)
    minResult['min_result'] = minResult.loc[:, ['result', 'min_result']].min(axis=1)
    minResult = minResult[min_columns]

mergeMinimums(is_greedy & is_closest)
mergeMinimums(is_greedy & is_furthest)
mergeMinimums(is_local & is_2opt)
mergeMinimums(is_local & is_swap)

def getPrecision(data_filter, name):
    tmp = pd.merge(df_result[data_filter][ext_columns], minResult, on=merge_cols)
    tmp = tmp[tmp["result"] != np.inf]
    tmp["precision"] = tmp["result"] / tmp["min_result"]
    tmp["name"] = name
    return tmp

prec_greedy_closest = getPrecision(is_greedy & is_closest, 'greedy closest first')
prec_greedy_furthest = getPrecision(is_greedy & is_furthest, 'greedy furthest first')
prec_local_2opt = getPrecision(is_local & is_2opt, 'local, 2opt')
prec_local_swap = getPrecision(is_local & is_swap, 'local, swap')
prec_grasp = getPrecision(is_grasp, 'grasp')

prec_all = prec_greedy_closest.append(prec_greedy_furthest
                             ).append(prec_local_2opt).append(prec_local_swap
                             ).append(prec_grasp)

In [ ]:
sns.boxplot(x='name', y='precision', data=prec_all)
plt.savefig(GRAPH_DIR + 'precision-big-all.png')
plt.show()

In [ ]:
sns.boxplot(x='nodes', y='precision', data=prec_local_2opt)
plt.savefig(GRAPH_DIR + 'precision-big-local-2opt.png')
plt.show()

In [ ]:
sns.boxplot(x='nodes', y='precision', data=prec_local_swap)
plt.savefig(GRAPH_DIR + 'precision-big-local-swap.png')
plt.show()

In [ ]:
sns.boxplot(x='nodes', y='precision', data=prec_grasp)
plt.savefig(GRAPH_DIR + 'precision-big-grasp.png')
plt.show()

In [ ]:
prec_grasp["precision"].describe()